In [ ]:
import pandas as pd
from math import radians, cos, sqrt

In [1]:
x = 1

In [ ]:
def udaljenost_izmedju_koordinata(koordinate1, koordinate2):
    R = 6371.0

    sirina1_rad = radians(koordinate1[0])
    duzina1_rad = radians(koordinate1[1])
    sirina2_rad = radians(koordinate2[0])
    duzina2_rad = radians(koordinate2[1])

    razlika_sirina = sirina2_rad - sirina1_rad
    razlika_duzina = duzina2_rad - duzina1_rad

    x = razlika_duzina * cos((sirina1_rad + sirina2_rad) / 2)
    y = razlika_sirina
    distanca = sqrt(x**2 + y**2) * R

    return distanca

In [ ]:
import ast
df = pd.read_csv("stajalista_sredjeno.csv")

lista_listi = df['Lista stanica'].apply(lambda x: ast.literal_eval(x))


distanca = udaljenost_izmedju_koordinata(lista_listi[0][0], lista_listi[0][1])

print(f"Distanca: {distanca} km.")

In [ ]:
dfTackica = pd.read_csv("linije_sredjeno.csv")

lista_tackica = dfTackica['Lista skretanja'].apply(lambda x: ast.literal_eval(x))


In [ ]:
cnt = 0
mapiranje = {}
for line in lista_listi:
    tackice = lista_tackica[cnt]
    for stanica in line:
        min_dist = 100000000
        min_tacka = None
        for tacka in tackice:
            dist = udaljenost_izmedju_koordinata(stanica, tacka)
            if dist < min_dist:
                min_dist = dist
                min_tacka = tacka
        mapiranje[min_tacka] = stanica
    cnt += 1

print(mapiranje)

In [ ]:
lista_stajalista_u_poretku = {}
lista_linija = df['ID linije'].tolist()
cnt = 0
for linija in lista_tackica:
    lista_stajalista_u_poretku[lista_linija[cnt]] = []
    for tacka in linija:
        if tacka in mapiranje:
            lista_stajalista_u_poretku[lista_linija[cnt]].append(mapiranje[tacka])
    cnt += 1


In [ ]:
print(lista_stajalista_u_poretku['1A'])


In [ ]:
def promeni_stajalista(red):
    red['Lista stanica'] = str(lista_stajalista_u_poretku[red['ID linije']])
    return red
df.apply(lambda x: promeni_stajalista(x), axis=1)

In [ ]:
df.to_csv("stajalista_u_poretku.csv", index=False)